In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import time
import matplotlib

from matplotlib.backends.backend_pdf import PdfPages
from skimage.feature import peak_local_max
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.special import gamma

In [2]:
x_cover = np.round(np.arange(-6.65, 6.66, 0.7), 2)
y_cover = np.round(np.arange(-6.65, 6.66, 0.7), 2)

def GetCarpetSignal(hist):
    return hist[hist.shape[0] // 2 - 10:hist.shape[0] // 2 + 10, hist.shape[0] // 2 - 10:hist.shape[0] // 2 + 10]

def VEM(hist):
    
    filt = (hist < 0.5)
    hist[filt] = 0.001
    hist = (1 + np.log(hist/8)/np.log(1.12)) // 1
    hist[hist < 0] = 0
    #hist[filt] = -1

    return hist

def neighbors(matrix, rowNumber, colNumber):
    result = []
    for rowAdd in range(-1, 2):
        newRow = rowNumber + rowAdd
        if newRow >= 0 and newRow <= len(matrix)-1:
            for colAdd in range(-1, 2):
                newCol = colNumber + colAdd
                if newCol >= 0 and newCol <= len(matrix)-1:
                    if newCol == colNumber and newRow == rowNumber:
                        continue
                    result.append(matrix[newCol][newRow])
    return np.array(result)

def find_smoothest_max(signal, coords):

    if (len(coords) == 0):
        return np.array([0, 0])

    decrease = np.zeros(len(coords))
    
    for i, coordinate in enumerate(coords):

        amplitude = signal[coordinate[0], coordinate[1]]

        neighbours = neighbors(signal, coordinate[1], coordinate[0])

        decrease[i] = np.sum(neighbours) / (len(neighbours))

    smoothest_max_coord = coords[np.argmax(decrease)]

    return smoothest_max_coord

x = np.array([-21.29, -21.63, 17.5, 21.16])
y = np.array([21.68, -21.13, 9.34, -21.05])
z = 1700

c_norm = 0.3

x_sq = x**2
y_sq = y**2
xy = x*y

x_mean = x.mean()
y_mean = y.mean()
x_mean_sq = x_mean**2
y_mean_sq = y_mean**2
x_sq_mean = x_sq.mean()
y_sq_mean = y_sq.mean()
xy_mean = xy.mean()
xy_mean_sq = xy_mean**2

def get_PFA_theta(t):
    
    t_mean = t.mean()
    xt_mean = (x*t).mean()
    yt_mean = (y*t).mean()

    xt_dif = xt_mean - x_mean*t_mean
    x2t_dif = x_sq_mean*t_mean - x_mean*xt_mean
    x2x_dif = x_mean_sq - x_sq_mean

    yt_dif = yt_mean - y_mean*t_mean
    y2t_dif = y_sq_mean*t_mean - y_mean*yt_mean
    y2y_dif = y_mean_sq - y_sq_mean

    nx = (xy_mean*yt_dif+x_mean*y2t_dif+xt_mean*y2y_dif)/(x_sq_mean*y_mean_sq+x_mean_sq*y_sq_mean-2*x_mean*y_mean*xy_mean+xy_mean_sq-x_sq_mean*y_sq_mean)*c_norm
    ny = (xy_mean*xt_dif+y_mean*x2t_dif+yt_mean*x2x_dif)/(x_sq_mean*y_mean_sq+x_mean_sq*y_sq_mean-2*x_mean*y_mean*xy_mean+xy_mean_sq-x_sq_mean*y_sq_mean)*c_norm

    nz = np.sqrt(1-nx**2-ny**2)

    theta = np.arccos(nz)
    
    if np.isnan(theta):
        return (-1, -1, -1, -1)
    else:
        return (np.degrees(theta), nx, ny, nz)

sigma_to = 2.6
b = 1.5
r_t = 30

def Chi_sq(params, x0, y0, t):
    
    nx, ny, nz, t0 = params

    chi_sq = 0

    n = len(t)
    
    for i in range(n):
        r_i = np.sqrt((x[i]-x0)**2+(y[i]-y0)**2)
        sigma_i = sigma_to*(1+r_i/r_t)**b
        w_i = 1/(sigma_i**2)

        chi_sq += w_i*(nx*x[i]+ny*y[i]*nz*z-c_norm*(t[i]-t0))**2
        
    return chi_sq

def constr(pars):
    nx, ny, nz, t0 = pars

    return nx**2+ny**2+nz**2-1
constraint = {'type': 'eq', 'fun': constr}

def get_PFAWTC_theta(t, x0, y0):

    PFA_result = get_PFA_theta(t)

    if (PFA_result == (-1, -1, -1, -1)):
        return [-1, -1]

    initial_guess = PFA_result

    result = minimize(Chi_sq, initial_guess, args=(float(x0), float(y0), t), constraints=constraint, tol=1e-6)

    theta = np.arccos(result.x[2])
    phi = (1 - np.sign(result.x[0]))*np.pi/2 + (1 + np.sign(result.x[0]))*(1 - np.sign(result.x[1]))*np.pi/2 + np.arctan(result.x[1]/result.x[0])
    phi_moved = np.radians(242) - phi
    if phi_moved < 0:
        phi_moved += 2*np.pi

    return [theta, phi]

def get_xy(signal):

    coordinates = peak_local_max(signal, exclude_border=False, threshold_rel=0.1)
    i, j = find_smoothest_max(signal, coordinates)

    x = x_cover[j]
    y = y_cover[19 - i]

    return [x, y]

def get_rho(i):

    signal = VEM(cover_signals[i])

    #x0, y0 = get_xy(signal)

    #theta, phi = get_PFAWTC_theta(time_array[i], x0, y0)

    rel_p = 8*1.12**(signal - 1)
    rel_p[rel_p < 8] = 0

    rho = np.zeros((20, 20))

    for i in range(len(y_cover)):
        for j in range(len(x_cover)):

            #x = x_cover[j]
            #y = y_cover[19 - i]
            
            #r_ij = np.sqrt(((x-x0)*np.sin(theta)*np.sin(phi) - (y-y0)*np.cos(theta)*np.cos(phi))**2 + ((x-x0)*np.cos(theta))**2 + ((y-y0)*np.cos(theta))**2)
            
            #k_ij = 1 + 7.503 / (1.636 + r_ij**1.474)
            rho_ij = rel_p[i, j] / 0.49

            if rho_ij > 0:

                rho[j, 19-i] = rho_ij

    return rho

def NKG(x, s, Ne):
    return (Ne / 95**2) * (gamma(4.5 - s)/(2*np.pi*gamma(s)*gamma(4.5 - 2*s))) * (x / 95)**(s - 2) * (1 + (x/95))**(s - 4.5)

def NKG_Vik(x, Ne, s): 
    return ((Ne/(95**2))*((5.803**(-(s-1.26)**2))/2.26)*(x/95)**(s-2)*(1+x/95)**(s-4.5))

def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]

In [3]:
path = 'proton_data/'

filenames = [path + filename for filename in os.listdir(path)]
filenames.sort(key=num_sort)